In [1]:
!pip install pysam
import pysam
import numpy as np
import pandas as pd
import gzip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 17.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

bam_reads_file = '/content/drive/My Drive/out_sort.bam'
bam_reads_index_file = '/content/drive/My Drive/out_sort.bam.bai'
reference_chr_file = '/content/drive/My Drive/chr1_1e6_2e6.fasta'
raw_reads_1_file = '/content/drive/My Drive/out.bwa.read1.fastq.gz'
raw_reads_2_file = '/content/drive/My Drive/out.bwa.read2.fastq.gz'
snps_file = '/content/drive/My Drive/putatative_snps.tsv'

snps = pd.read_csv('/content/drive/My Drive/putatative_snps.tsv', sep='\t')

snps.head()

# Open BAM file
bam_file = pysam.AlignmentFile('/content/drive/My Drive/out_sort.bam', "rb")

# Open FASTA file (if needed for reference sequences)
fasta_file = pysam.FastaFile('/content/drive/My Drive/chr1_1e6_2e6.fasta')


Mounted at /content/drive


In [3]:
import pandas as pd
import pysam
import numpy as np

def main():
    from google.colab import drive
    drive.mount('/content/drive')

    bam_reads_file = '/content/drive/My Drive/out_sort.bam'
    snps_file = '/content/drive/My Drive/putatative_snps.tsv'

    # Load SNP data
    snps = pd.read_csv(snps_file, sep='\t')

    # Open BAM file
    bam_file = pysam.AlignmentFile(bam_reads_file, "rb")

    # Function to get read counts and quality scores
    def get_read_counts_and_quality_scores(bam_file, chrom, pos):
        n_ref = 0
        n_alt = 0
        qual_scores = []
        for pileupcolumn in bam_file.pileup(chrom, pos-1, pos):
            if pileupcolumn.pos == pos-1:
                for pileupread in pileupcolumn.pileups:
                    if not pileupread.is_del and not pileupread.is_refskip:
                        base = pileupread.alignment.query_sequence[pileupread.query_position]
                        ref_base = row['ref']
                        alt_base = row['alt']
                        if base == ref_base:
                            n_ref += 1
                        elif base == alt_base:
                            n_alt += 1
                        qual_scores.append(pileupread.alignment.query_qualities[pileupread.query_position])
        error_rate = np.mean([10 ** (-q / 10) for q in qual_scores]) if qual_scores else 0.01
        return n_ref, n_alt, error_rate

    # Bayesian calculation of posteriors
    def calculate_posterior(n_ref, n_alt, ref, alt, maf, error_rate):
        # Prior probabilities based on Hardy-Weinberg Equilibrium
        P_AA = (1 - maf) ** 2
        P_AB = 2 * maf * (1 - maf)
        P_BB = maf ** 2

        # Likelihoods with error rates
        P_data_given_AA = ((1 - error_rate) ** n_ref) * (error_rate ** n_alt)
        P_data_given_AB = 0.5 ** (n_ref + n_alt)
        P_data_given_BB = ((1 - error_rate) ** n_alt) * (error_rate ** n_ref)

        # Bayes' Theorem
        P_data = P_data_given_AA * P_AA + P_data_given_AB * P_AB + P_data_given_BB * P_BB
        return {
            ref+ref: P_data_given_AA * P_AA / P_data,
            ref+alt: P_data_given_AB * P_AB / P_data,
            alt+alt: P_data_given_BB * P_BB / P_data
        }

    results = []
    for index, row in snps.iterrows():
        chrom = row['chr']
        pos = int(row['pos'])
        maf = row['maf'] if not pd.isna(row['maf']) else 0.5

        n_ref, n_alt, error_rate = get_read_counts_and_quality_scores(bam_file, chrom, pos)
        posteriors = calculate_posterior(n_ref, n_alt, row['ref'], row['alt'], maf, error_rate)
        results.append({
            'chromosome': chrom,
            'position': pos,
            'putative_genotype': max(posteriors, key=posteriors.get),
            'posterior_probability': max(posteriors.values()),
            'n_reads': n_ref + n_alt
        })

    results_df = pd.DataFrame(results)
    results_csv_path = results_df.to_csv('/content/drive/My Drive/snp_caller_results9.csv', index=False)
    print(results_df.head())

    bam_file.close()

if __name__ == '__main__':
    main()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
             chromosome  position putative_genotype  posterior_probability  \
0  chr1:1000000-2000000    172741                AA               0.902500   
1  chr1:1000000-2000000    325026                CC               0.688900   
2  chr1:1000000-2000000    375797                AA               0.894754   
3  chr1:1000000-2000000    423797                TA               0.836288   
4  chr1:1000000-2000000    518726                CG               0.501136   

   n_reads  
0        0  
1        0  
2        1  
3        2  
4        1  


In [4]:
    from google.colab import drive
    drive.mount('/content/drive')

    # Define file paths
    bam_reads_file = '/content/drive/My Drive/out_sort.bam'
    snps_file = '/content/drive/My Drive/putatative_snps.tsv'

    # Load SNP data from a TSV file
    snps = pd.read_csv(snps_file, sep='\t')

    # Open the BAM file using pysam
    bam_file = pysam.AlignmentFile(bam_reads_file, "rb")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pysam
import pandas as pd

def count_reads_at_positions(bam_file_path, positions):
    """
    Count reads in a BAM file at specified positions.

    Parameters:
        bam_file_path (str): Path to the BAM file.
        positions (list of tuples): List of tuples where each tuple is (chromosome, position).

    Returns:
        dict: Dictionary with keys as (chromosome, position) and values as read counts.
    """
    # Open the BAM file
    bam_file = pysam.AlignmentFile(bam_file_path, "rb")
    read_counts = {}

    # Iterate through each specified position
    for chrom, pos in positions:
        count = 0
        # Fetch reads overlapping the position (0-based indexing adjustment)
        for pileupcolumn in bam_file.pileup(chrom, pos-1, pos, truncate=True):
            if pileupcolumn.pos == pos - 1:
                count = pileupcolumn.nsegments
        read_counts[(chrom, pos)] = count

    # Close the BAM file
    bam_file.close()
    return read_counts

# Example usage
if __name__ == "__main__":
    positions_to_check = [("chr1:1000000-2000000", 172741), ("chr1:1000000-2000000", 325026), ("chr1:1000000-2000000", 375797)]  # Add your positions here

    # Path to your BAM file
    bam_file_path = '/content/drive/My Drive/out_sort.bam'

    # Get read counts
    counts = count_reads_at_positions(bam_file_path, positions_to_check)

    # Print results
    for pos, count in counts.items():
        print(f"Position {pos}: {count} reads")


Position ('chr1:1000000-2000000', 172741): 5 reads
Position ('chr1:1000000-2000000', 325026): 3 reads
Position ('chr1:1000000-2000000', 375797): 8 reads


In [8]:
import pysam
import pandas as pd

def verify_read_counts(bam_file_path, positions, ref_alt_pairs):
    """
    Verifies read counts at specified positions using pysam and compares with expected counts.

    Parameters:
        bam_file_path (str): Path to the BAM file.
        positions (list): List of positions (chromosome, position) to check.
        ref_alt_pairs (dict): Dictionary with (chromosome, position) as key and (ref, alt) as value.

    Returns:
        None: Prints the results directly.
    """
    bam_file = pysam.AlignmentFile(bam_file_path, "rb")

    for chrom, pos in positions:
        n_ref, n_alt = 0, 0
        ref, alt = ref_alt_pairs[(chrom, pos)]

        for pileupcolumn in bam_file.pileup(chrom, pos-1, pos, truncate=True):
            if pileupcolumn.pos == pos - 1:
                for pileupread in pileupcolumn.pileups:
                    if not pileupread.is_del and not pileupread.is_refskip:
                        base = pileupread.alignment.query_sequence[pileupread.query_position]
                        if base == ref:
                            n_ref += 1
                        elif base == alt:
                            n_alt += 1

        print(f"Position: {chrom}:{pos}, Reference (Ref): {ref}, Alternate (Alt): {alt}")
        print(f"Calculated n_ref: {n_ref}, n_alt: {n_alt}")
        print("----------")

    bam_file.close()

# Example usage
if __name__ == "__main__":
    bam_path = '/content/drive/My Drive/out_sort.bam'
    # Example data - replace with your actual data
    positions = [("chr1:1000000-2000000", 172741), ("chr1:1000000-2000000", 325026), ("chr1:1000000-2000000", 375797), ("chr1:1000000-2000000", 423797), ("chr1:1000000-2000000", 518726), ("chr1:1000000-2000000", 568632), ("chr1:1000000-2000000", 868896)]  # List of tuples (chromosome, position)
    ref_alt_pairs = {
        ("chr1:1000000-2000000", 172741): ("A", "G"),
        ("chr1:1000000-2000000", 325026): ("T", "C"),
        ("chr1:1000000-2000000", 375797): ("A", "T"),
        ("chr1:1000000-2000000", 423797): ("T", "A"),
        ("chr1:1000000-2000000", 518726): ("C", "G"),
        ("chr1:1000000-2000000", 568632): ("A", "T"),
        ("chr1:1000000-2000000", 868896): ("T", "A")

    }

    verify_read_counts(bam_path, positions, ref_alt_pairs)


Position: chr1:1000000-2000000:172741, Reference (Ref): A, Alternate (Alt): G
Calculated n_ref: 0, n_alt: 0
----------
Position: chr1:1000000-2000000:325026, Reference (Ref): T, Alternate (Alt): C
Calculated n_ref: 0, n_alt: 0
----------
Position: chr1:1000000-2000000:375797, Reference (Ref): A, Alternate (Alt): T
Calculated n_ref: 1, n_alt: 0
----------
Position: chr1:1000000-2000000:423797, Reference (Ref): T, Alternate (Alt): A
Calculated n_ref: 0, n_alt: 2
----------
Position: chr1:1000000-2000000:518726, Reference (Ref): C, Alternate (Alt): G
Calculated n_ref: 0, n_alt: 1
----------
Position: chr1:1000000-2000000:568632, Reference (Ref): A, Alternate (Alt): T
Calculated n_ref: 0, n_alt: 0
----------
Position: chr1:1000000-2000000:868896, Reference (Ref): T, Alternate (Alt): A
Calculated n_ref: 2, n_alt: 0
----------


## Main Methods:

1. **Mounting Google Drive**:
   - This is essential for accessing files stored in Google Drive when running the script in Google Colab. It sets up the file system to access BAM files, reference files, and SNP data.

2. **Reading SNP Data**:
   - The script reads a tab-separated values (TSV) file containing SNP data into a pandas DataFrame. This is critical for fetching the SNP positions to be analyzed.

3. **Opening Files with `pysam`**:
   - **BAM File**: Opens a BAM file for reading aligned sequencing data. This is used to fetch alignment data at specific genomic positions.
   - **FASTA File**: Optionally, a FASTA file is opened if reference sequences are needed for comparison or other analysis.

4. **get_read_counts_and_quality_scores**:
   - This function retrieves the number of reads aligning to reference (ref) and alternate (alt) alleles at specific positions within the genome. It also calculates the average error rate based on quality scores. This is crucial for understanding the coverage and quality of sequencing at positions of interest.

5. **calculate_posterior**:
   - Computes the posterior probabilities for genotype assignments based on observed allele counts, using a Bayesian model with priors based on Hardy-Weinberg Equilibrium. This method is central to determining the most likely genotypes at SNP positions.

6. **Data Analysis and Export**:
   - The script iterates through SNP positions, using the above functions to calculate read counts, error rates, and posterior probabilities. It then assembles these results into a DataFrame and exports this to a CSV file. This summarized data provides insights into potential genotypes and the reliability of these determinations based on sequencing data.

### Auxiliary Functions:

- **count_reads_at_positions**: Counts reads at specified genomic positions. Useful for preliminary checks of read coverage.
- **verify_read_counts**: Compares observed read counts to expected counts based on reference and alternate alleles, providing a straightforward method to validate sequencing and alignment accuracy.

### Main Results:

The script outputs a DataFrame that lists SNPs along with calculated metrics such as the most likely genotype, its posterior probability, and the number of reads supporting each allele. This DataFrame is saved to a CSV file for further analysis.

